# Introduction
Analysis of representation of languages in NYC

# Setup

## Dependencies

In [ ]:
%pip install pandas==2.3.1 pandasql==0.7.3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sqlalchemy-2.0.41-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
Using cached sqlalchemy-2.0.41-cp313-cp313-macosx_11_0_arm64.whl (2.1 MB)
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26867 sha256=2e720bebb9c23eca0fb952e97913bb4f1788e27ceba26e253584723d3ef2f322
  Stored in directory: /Users/alex.bieniek/Library/Caches/pip/wheels/b4/d0/8c/a6b366870bf041849cd96e03b71641e082f8d6456269b603b7
Successfully built pandasql
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandasql]1/3 [sqlalchemy]
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import pandasql as ps

## Data Collection
Save the following data dumps as described

- NYC OpenData
    - [LEP Presence by Language](https://data.cityofnewyork.us/City-Government/Population-and-Languages-of-the-Limited-English-Pr/ajin-gkbp/about_data) -> `data/lep.csv`

In [59]:
lep = pd.read_csv(
    'data/lep.csv',
    names=['years', 'borough', 'districtCode', 'district', 'language', 'lepPopulation', 'lepPercent', 'cvalepPopulation', 'cvalepPercent'],
    header=0,
    quotechar='"',
    delimiter=','
)
lep.Name = "lep"
lep.describe(include='all')

,years,borough,districtCode,district,language,lepPopulation,lepPercent,cvalepPopulation,cvalepPercent
count,8024,8024,8024.000000,8024,8024,8024.000000,8024.000000,8024.000000,8024.000000
unique,1,5,NaN,59,136,NaN,NaN,NaN,NaN
top,2015-2019,Brooklyn,NaN,"Battery Park City, Tribeca",Afrikaans,NaN,NaN,NaN,NaN
freq,8024,2448,NaN,136,59,NaN,NaN,NaN,NaN
mean,NaN,NaN,280.305085,NaN,NaN,221.569167,0.735070,111.514706,0.735456
std,NaN,NaN,117.765530,NaN,NaN,2122.544665,5.380983,992.670240,5.438624
min,NaN,NaN,101.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,203.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,306.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,403.000000,NaN,NaN,14.000000,0.000000,0.000000,0.000000


# Analysis

In [ ]:
# Districts by Borough
# Note: districts mention multiple neighborhoods
districts_by_borough_df = ps.sqldf(f"""
    SELECT
        json_group_array(distinct json_object('district', district)) AS districts,
        borough
    FROM {lep.Name}
    GROUP BY borough
""")

import json
for _, districts, borough in districts_by_borough_df.to_records():
    print(
        f"{borough}: \n{'\n'.join("\t"+d['district'] for d in json.loads(districts))}"
    )

Bronx: 
	Melrose, Mott Haven, Port Morris
	Hunts Point, Longwood
	Morrisania, Crotona Park East
	Highbridge, Concourse Village
	University Hts., Fordham, Mt. Hope
	East Tremont, Belmont
	Bedford Park, Norwood, Fordham
	Riverdale, Kingsbridge, Marble Hill
	Soundview, Parkchester
	Throgs Nk., Co-op City, Pelham Bay
	Pelham Pkwy, Morris Park, Laconia
	Wakefield, Williamsbridge
Brooklyn: 
	Williamsburg, Greenpoint
	Brooklyn Heights, Fort Greene
	Bedford Stuyvesant
	Bushwick
	East New York, Starrett City
	Park Slope, Carroll Gardens
	Sunset Park, Windsor Terrace
	Crown Heights North
	Crown Heights South, Wingate
	Bay Ridge, Dyker Heights
	Bensonhurst, Bath Beach
	Borough Park, Ocean Parkway
	Coney Island, Brighton Beach
	Flatbush, Midwood
	Sheepshead Bay, Gerritsen Beach
	Brownsville, Ocean Hill
	East Flatbush, Rugby, Farragut
	Canarsie, Flatlands
Manhattan: 
	Battery Park City, Tribeca
	Greenwich Village, Soho
	Lower East Side, Chinatown
	Chelsea, Clinton
	Midtown Business District
	Stuyve

In [87]:
ps.sqldf(f"""
    SELECT language, SUM(cvalepPopulation)
    FROM {lep.Name}
    GROUP BY language
""")

,language,SUM(cvalepPopulation)
0,Afrikaans,0
1,Akan (incl. Twi),2661
2,Albanian,7048
3,Aleut languages,21
4,Amharic,516
...,...,...
131,Vietnamese,4553
132,Wolof,183
133,Yiddish,13360
134,Yoruba,1060
